In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
from tqdm import tqdm
tqdm.pandas()

In [3]:
os.chdir("C:/Users/vikas/Downloads/txt_reviews/txt_reviews")

In [4]:
folder_path = "C:/Users/vikas/Downloads/txt_reviews/txt_reviews"
file_list = glob.glob(folder_path+"/*.txt")
main_datframe = pd.DataFrame(pd.read_table(file_list[0],sep=":"))

lst = []
ProductId = []
UserId = []
ProfileName = []
HelpfulnessNumerator = []
HelpfulnessDenominator = []
Score = []
Time = []
ReviewSummary = []
ReviewText = []

for i in tqdm(range(1,len(file_list)+1)):
    with open("review_"+str(i)+".txt","r",encoding='utf-8') as f:
        data = f.read()
        lst.append(data.split("\n"))
        
for i in lst:
    ProductId.append(i[0].split()[1])
    UserId.append(i[1].split()[1])
    ProfileName.append(i[2].split()[1])
    HelpfulnessNumerator.append(i[3].split()[1])
    HelpfulnessDenominator.append(i[4].split()[1])
    Score.append(i[5].split()[1])
    Time.append(i[6].split()[1])
    ReviewSummary.append(i[7].split(":")[1])
    ReviewText.append(i[8].split(":")[1])

100%|████████████████████████████████████████████████████████████████████████| 568454/568454 [04:43<00:00, 2005.34it/s]


In [5]:
df_before_cleaning = pd.DataFrame({"product_id":ProductId,"user_id":UserId,"profilename":ProfileName,"helpfulness_number":HelpfulnessNumerator,"helpfulness_denominator":HelpfulnessDenominator,"score":Score,"time":Time,"review_summay":ReviewSummary,"Review_text":ReviewText})

In [7]:
df_before_cleaning

,product_id,user_id,profilename,helpfulness_number,helpfulness_denominator,score,time,review_summay,Review_text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned ...
1,B00813GRG4,A1D87F6ZCVE5NK,dll,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanu...
2,B000LQOCH0,ABXLMWJIXXAIN,Natalia,1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a f...
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient ...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,Michael,0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wi...
...,...,...,...,...,...,...,...,...,...
568449,B001EO7N10,A28KG5XORO54AY,Lettie,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if n...
568450,B003S1WTCU,A3I8AFVPEE8KI5,R.,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocola...
568451,B004I613EE,A121AA1GQV751Z,pksd,2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 ..."
568452,B004I613EE,A3IBEVCTXKNOH,Kathy,1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and re...


In [9]:
df_before_cleaning[df_before_cleaning.Review_text.isnull() == True]

,product_id,user_id,profilename,helpfulness_number,helpfulness_denominator,score,time,review_summay,Review_text


In [15]:
df_before_cleaning.to_csv("df_before_cleaning.csv",index=False)

In [5]:
df = pd.read_csv("df_before_cleaning.csv")

In [17]:
df.Review_text.isnull().sum()

0

In [19]:
x = df[["Review_text"]]
y = df[["score"]]

# Train, Test split

In [31]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.75,random_state=50)

In [32]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [33]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# preprocess

In [34]:
def preprocess(raw_text,flag):
    
    # Remove Special Characters
    sentence = re.sub("[^a-zA-z]"," ",raw_text)
    
    # Convert to Lower Case
    sentence = sentence.lower()
    
    # Tokenizing the sentence
    tokens = sentence.split()
    
    # Remove StopWords
    clean_tokens = [t for t in tokens if not t in stopwords.words("english")]
    
    # Stemming / Lemmatizing
    if flag == "stem" :
        clean_tokens = [stemmer.stem(word) for word in clean_tokens]
    else:
        clean_tokens = [lemmatizer.lemmatize(word) for word in clean_tokens]
        
    return pd.Series([" ".join(clean_tokens),len(clean_tokens)])

In [35]:
x_train

,Review_text
195720,THIS is my morning addiction--not coffee. I'...
555934,"Just left packaging feedback, but upon review..."
99591,Way too salty has an orange hue taste that is...
160146,Taste very similar to Lipton's loose leaf tea...
483074,I think it might be good cat food but my cats...
...,...
385156,"I must have somehow ""missed"" that this was de..."
321502,"Very smooth, never bitter. Wonderful for me b..."
441633,I was a big fan of Natural Balance because of...
239499,These cookies are ridiculous. I don't know wh...


# x_train_preprocess

In [29]:
x_train_df = x_train["Review_text"].progress_apply(lambda x : preprocess(x,"stem"))

x_train_df.head()

 36%|█████████████████████████▍                                            | 144806/397917 [1:30:19<2:37:52, 26.72it/s]


KeyboardInterrupt: 

In [ ]:
x_train_df.columns = ["clean_review_text","text_length"]

In [4]:
x_train_df = pd.read_csv("C:/Users/vikas/Downloads/txt_reviews/txt_reviews/x_train_df")

x_train_df

,Unnamed: 0,Review_text,len_text
0,195720,morn addict coffe drink green tea year tri man...,31
1,555934,left packag feedback upon review issu appear q...,56
2,99591,way salti orang hue tast discusst bought obvio...,16
3,160146,tast similar lipton loos leaf tea prefer kind ...,18
4,483074,think might good cat food cat love alway left ...,20
...,...,...,...
426335,385156,must somehow miss design pod brew system thoug...,102
426336,321502,smooth never bitter wonder low acid coffe prob...,12
426337,441633,big fan natur balanc grain free formula small ...,25
426338,239499,cooki ridicul know kind chocol chunk got riche...,53


In [5]:
x_train_df.drop("Unnamed: 0",axis = 1, inplace = True)

In [6]:
x_train_df

,Review_text,len_text
0,morn addict coffe drink green tea year tri man...,31
1,left packag feedback upon review issu appear q...,56
2,way salti orang hue tast discusst bought obvio...,16
3,tast similar lipton loos leaf tea prefer kind ...,18
4,think might good cat food cat love alway left ...,20
...,...,...
426335,must somehow miss design pod brew system thoug...,102
426336,smooth never bitter wonder low acid coffe prob...,12
426337,big fan natur balanc grain free formula small ...,25
426338,cooki ridicul know kind chocol chunk got riche...,53


In [7]:
y_train_df = pd.read_csv("C:/Users/vikas/Downloads/txt_reviews/txt_reviews/y_train_df")
y_train_df.drop("Unnamed: 0",axis = 1, inplace = True)
y_train_df

,score
0,5
1,1
2,1
3,2
4,4
...,...
426335,3
426336,5
426337,1
426338,5


In [8]:
temp_train_df = pd.concat([x_train_df,y_train_df],axis = 1)
temp_train_df.drop(temp_train_df[temp_train_df.Review_text.isnull() == True].index, inplace = True)

In [9]:
x_train_df = temp_train_df[["Review_text","len_text"]]

In [10]:
y_train_df = temp_train_df[["score"]]

In [11]:
x_train_df.head(3)

,Review_text,len_text
0,morn addict coffe drink green tea year tri man...,31
1,left packag feedback upon review issu appear q...,56
2,way salti orang hue tast discusst bought obvio...,16


In [12]:
y_train_df.head(3)

,score
0,5
1,1
2,1


# x_test_preprocess

In [13]:
x_test_df = x_test["Review_text"].progress_apply(lambda x : preprocess(x,"stem"))

x_test_df.head()

NameError: name 'x_test' is not defined

In [ ]:
x_test_df.columns = ["clean_review_text","text_length"]

In [15]:
x_test_df = pd.read_csv("C:/Users/vikas/Downloads/txt_reviews/txt_reviews/x_test_df")

x_test_df

,Unnamed: 0,0,1
0,111074,izz great tast refresh beverag percent fruit j...,134
1,99817,bought berri nut mix look tasti instead percie...,42
2,353268,powder good contain horribl tap top clear disp...,13
3,487407,soup great bit salti like otherwis nice tast p...,14
4,178080,dog gobbl food look realli fresh healthi actua...,13
...,...,...,...
142109,219060,gave gift chocol lover love actual hid everybo...,26
142110,136928,best cake mix found well know may find better,9
142111,232091,well product good residu tag antler care want ...,25
142112,524998,know cooki good sort bland tast littl cannot d...,42


In [16]:
x_test_df.drop("Unnamed: 0",axis = 1, inplace = True)

In [17]:
x_test_df.columns = ["clean_review_text","text_length"]

In [18]:
x_test_df.head(1)

,clean_review_text,text_length
0,izz great tast refresh beverag percent fruit j...,134


In [19]:
y_test_df = pd.read_csv("C:/Users/vikas/Downloads/txt_reviews/txt_reviews/y_test_df")
y_test_df.drop("Unnamed: 0",axis = 1, inplace = True)
y_test_df

,score
0,4
1,4
2,3
3,3
4,5
...,...
142109,5
142110,4
142111,3
142112,1


In [20]:
temp_test_df = pd.concat([x_test_df,y_test_df],axis = 1)
temp_test_df.drop(temp_test_df[temp_test_df.clean_review_text.isnull() == True].index, inplace = True)

In [21]:
x_test_df = temp_test_df[["clean_review_text","text_length"]]

In [22]:
y_test_df = temp_test_df[["score"]]

In [23]:
x_test_df

,clean_review_text,text_length
0,izz great tast refresh beverag percent fruit j...,134
1,bought berri nut mix look tasti instead percie...,42
2,powder good contain horribl tap top clear disp...,13
3,soup great bit salti like otherwis nice tast p...,14
4,dog gobbl food look realli fresh healthi actua...,13
...,...,...
142109,gave gift chocol lover love actual hid everybo...,26
142110,best cake mix found well know may find better,9
142111,well product good residu tag antler care want ...,25
142112,know cooki good sort bland tast littl cannot d...,42


In [24]:
y_test_df

,score
0,4
1,4
2,3
3,3
4,5
...,...
142109,5
142110,4
142111,3
142112,1


# Bow 

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

vocab = CountVectorizer()

x_train_bow = vocab.fit_transform(x_train_df["Review_text"])

In [26]:
x_test_bow = vocab.transform(x_test_df["clean_review_text"])

# logistics Regression

In [71]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(x_train_bow, y_train_df)

C:\Users\vikas\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vikas\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [70]:
y_test_pred = classifier.predict(x_test_bow)

In [73]:
from sklearn.metrics import accuracy_score, classification_report

print(accuracy_score(y_test_df, y_test_pred))

print(classification_report(y_test_df, y_test_pred))

0.7296399515806661
              precision    recall  f1-score   support

           1       0.65      0.64      0.65     13002
           2       0.42      0.21      0.28      7402
           3       0.46      0.27      0.34     10720
           4       0.51      0.23      0.31     20049
           5       0.78      0.95      0.86     90919

    accuracy                           0.73    142092
   macro avg       0.56      0.46      0.49    142092
weighted avg       0.69      0.73      0.69    142092



# RandomForest

In [79]:
!pip install keras

In [85]:
from tqdm.keras import TqdmCallback
cbk = TqdmCallback()

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

In [91]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(x_train_bow, y_train_df)

C:\Users\vikas\AppData\Local\Temp\ipykernel_27896\190730892.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(x_train_bow, y_train_df)


RandomForestClassifier()

In [92]:
y_test_pred_random_forest = classifier.predict(x_test_bow)

In [95]:
from sklearn import metrics

In [97]:
metrics.accuracy_score(y_test_df,y_test_pred_random_forest)

0.7939152098640317

# decision Tree

In [98]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(x_train_bow, y_train_df)

DecisionTreeClassifier()

In [100]:
y_test_pred_decision_tree = classifier.predict(x_test_bow)

In [101]:
metrics.accuracy_score(y_test_df,y_test_pred_decisin_tree)

0.7458196098302509

# KNN

In [102]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
classifier.fit(x_train_bosw,y_train_df)

C:\Users\vikas\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [103]:
y_test_pred_knn = classifier.predict(x_test_bow)

In [104]:
metrics.accuracy_score(y_test_df,y_test_pred_knn)

0.6665892520338935

In [29]:
y_test_df

,score
0,4
1,4
2,3
3,3
4,5
...,...
142109,5
142110,4
142111,3
142112,1


# Naive base algorithm

In [28]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train_bow, y_train_df)
  

C:\Users\vikas\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.